In [32]:
import pandas as pd
pd.set_option('display.max_columns', 300)
import matplotlib.pyplot as plt
from math import exp
from scipy import stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 300)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
import pickle
from sklearn.preprocessing import PolynomialFeatures

## Step 1: Read in hold out data, scalers, and best model

In [11]:
df = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [12]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [13]:
df.drop(columns=['date'],inplace = True)

In [34]:
final_model = pickle.load(open("model.pickle","rb"))

In [36]:
selected_columns = pickle.load(open("selected_columns.pickle","rb"))

In [6]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

**Bedroom cleaning**

In [15]:
df['bedrooms'].replace(to_replace=[11,33],value=[4,3],inplace=True)

df['bedrooms'] = np.where(df['id'] == 6306400140, 5, df['bedrooms'])

df['bedrooms'] = np.where(df['id'] == 2954400190, 4, df['bedrooms'])

**Bathroom cleaning**

In [17]:
df['bathrooms'] = np.where(df['id'] == 6306400140, 4.5, df['bathrooms'])

df['bathrooms'] = np.where(df['id'] == 2954400190, 4.5, df['bathrooms'])

**Years Old**

In [18]:
df['years_old'] = 2020- pd.DatetimeIndex(df['yr_built']).year

**New Home**

In [19]:
df['new_home'] = np.where(df['yr_built']>=2013, 1, 0)

**Renovated Home**

In [20]:
df['renovated_home'] = np.where(df['yr_renovated']>0, 1, 0)

**Converting Bedrooms to Dummy**

In [21]:
bedrooms = pd.get_dummies(df['bedrooms'])

In [22]:
df = pd.concat([df,bedrooms],axis=1)

In [23]:
df.drop(columns=[0],inplace = True)

In [24]:
df.rename(columns={1:'bed1',2:'bed2',3:'bed3',4:'bed4',5:'bed5',6:'bed6',7:'bed7',8:'bed8',9:'bed9',10:'bed10'},inplace=True)

**Location Variable**

In [26]:
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [27]:
df['dist_seattle'] = haversine(47.6062,-122.3321, df['lat'].values, df['long'].values)

**Making Polynomials**

In [28]:
features = ['bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'years_old',
 'new_home',
 'renovated_home',
 'bed1',
 'bed2',
 'bed3',
 'bed4',
 'bed5',
 'bed6',
 'bed7',
 'bed8',
 'bed9',
 'bed10',
 'dist_seattle']

In [29]:
df_features = df[features]

In [33]:
poly = PolynomialFeatures(degree=2, include_bias=False)

poly_data = poly.fit_transform(df_features)

poly_columns = poly.get_feature_names(df_features.columns)

df_poly = pd.DataFrame(poly_data, columns=poly_columns)

In [ ]:
# transformed_holdout = final_scaler(holdout)


Index(['sqft_living', 'grade', 'bathrooms grade', 'bathrooms sqft_living15',
       'sqft_living condition', 'sqft_living grade', 'sqft_living yr_built',
       'sqft_living zipcode', 'sqft_living lat', 'sqft_living long',
       'sqft_living sqft_living15', 'sqft_living years_old', 'grade^2',
       'grade sqft_above', 'grade yr_built', 'grade zipcode', 'grade lat',
       'grade long', 'grade sqft_living15', 'grade years_old'],
      dtype='object')

## Step 3: Predict the holdout set

In [38]:
y_pred = final_model.predict(df_poly[selected_columns])

## Step 4: Export your predictions

In [39]:
pd.DataFrame(y_pred).to_csv('housing_preds_alexandra_bruno.csv')